In [ ]:
"""Please keep in mind that this whole document is only
to give a general idea of what to implement. 
These are not functioning codes."""

In [1]:
%pip install "requests==2.28.1"

You should consider upgrading via the 'pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import sys
from urllib.parse import urljoin
import requests
sys.path.insert(1, '/Users/anureddy/Desktop/Sem01/DataScience_for_text_analytics/Project1/slang-translator/Connection')

from elastic_con import client 
import nltk


In [3]:
import spacy

In [ ]:
from gensim.parsing.preprocessing import remove_stopwords

In [31]:
#from elasticsearch import Elasticsearch

In [ ]:
#Not required as the connection has been taken care of 
# # test code for formality. Does not work on my laptop.
es_host = "http://localhost:9200"
es_user = "elastic"
es_password = "1234"

es = Elasticsearch(
    hosts=es_host,  # "http://localhost:9200"
    http_auth=(es_user, es_password)   # credentials for basic authentication
)

In [4]:
# draft : implementation step
######################################
# translate regular sentence to slang#
######################################

input_str = "I have something to say"

In [5]:
nlp = spacy.load("en_core_web_lg")

In [6]:
str_doc = nlp(input_str)

In [9]:
len(str_doc)

5

In [30]:
for np in str_doc.noun_chunks:
    print(np)
# print noun phrases.

I
something


In [32]:
"""To find and return subject of a preprocessed sentence."""
# I wrote the function based on which one may
# omit the subject of a sentence with a possibility. 
# This is not used now, as I still don't
# know how to calculate the weight of such omission.
def subject_find(doc):
    for token in doc:
        if "subj" in token.dep_:
            subtree = list(token.subtree)
            start = subtree[0].i
            end = subtree[-1].i + 1
            return doc[start:end]

In [23]:
# do slang dictionary query with part of the input sentence
"""returns all possible k-word-shingle for the input sentence.
form: list, [[1-shingle][2-shingle]...[k-shingle]]""" 
def multiple_shingle(doc_str):
    doc = doc_str.split(" ")
    shingle_lst=[]
    for k in range(0,len(doc)):
        shingle_lst_temp = []
        for i in range(len(doc)-k):
            shingle_lst_temp.append(doc[i:k+i+1])
        shingle_lst.append(shingle_lst_temp)
    return shingle_lst

In [24]:
multiple_shingle(input_str)

[[['I'], ['have'], ['something'], ['to'], ['say']],
 [['I', 'have'], ['have', 'something'], ['something', 'to'], ['to', 'say']],
 [['I', 'have', 'something'],
  ['have', 'something', 'to'],
  ['something', 'to', 'say']],
 [['I', 'have', 'something', 'to'], ['have', 'something', 'to', 'say']],
 [['I', 'have', 'something', 'to', 'say']]]

In [3]:
# ES search query to fetch all the data 
index_name = "slang-demo-json"
request = '''{"query": {"match_all": {}}}'''
results = client.search(index=index_name, body=request,size = 100)['hits']['hits']
print(results)

[{'_index': 'slang-demo-json', '_type': '_doc', '_id': '0', '_score': 1.0, '_source': {'id': 0, 'acronym': '2day', 'meaning': 'today'}}, {'_index': 'slang-demo-json', '_type': '_doc', '_id': '1', '_score': 1.0, '_source': {'id': 1, 'acronym': '2m2h', 'meaning': 'too much too handle'}}, {'_index': 'slang-demo-json', '_type': '_doc', '_id': '2', '_score': 1.0, '_source': {'id': 2, 'acronym': '2moro', 'meaning': 'tomorrow'}}, {'_index': 'slang-demo-json', '_type': '_doc', '_id': '3', '_score': 1.0, '_source': {'id': 3, 'acronym': '2nite', 'meaning': 'tonight'}}, {'_index': 'slang-demo-json', '_type': '_doc', '_id': '4', '_score': 1.0, '_source': {'id': 4, 'acronym': '4eae', 'meaning': 'for ever and ever'}}, {'_index': 'slang-demo-json', '_type': '_doc', '_id': '5', '_score': 1.0, '_source': {'id': 5, 'acronym': 'aaf', 'meaning': 'always and forever'}}, {'_index': 'slang-demo-json', '_type': '_doc', '_id': '6', '_score': 1.0, '_source': {'id': 6, 'acronym': 'aar', 'meaning': 'at any rate'}

/Users/anureddy/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:3: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


In [5]:
request = '''{"query": {"match": { "acronym":"2day"}}}'''
results = client.search(index=index_name, body=request,size = 100)['hits']['hits']
print(results)

[{'_index': 'slang-demo-json', '_type': '_doc', '_id': '0', '_score': 7.682827, '_source': {'id': 0, 'acronym': '2day', 'meaning': 'today'}}]


/Users/anureddy/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:2: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  


In [ ]:
"""search in slang database based on shingles,
return slang and the corresponding shingle.
NOTE: not finished. """
def slang_search(shingle_lst):
    lst_search_result = []
    for i in range(len(shingle_lst)):
        list_temp = shingle_lst[i]
        for j in range(len(list_temp)):
            search_result = client.search(index=index_name,query={"match": {
                              "expansion": list_temp[j]}})
            # TODO: make it a fuzzy search
            if search_result["hits"]["hits"]:
                for hit in search_result["hits"]["hits"]:
                    lst_search_result.append((list_temp[j],hit["abbreviation"]))
    return lst_search_result

In [4]:
  
index_name = "emo31"
request = '''{"query": {"match_all": {}}}'''
results = client.search(index=index_name, body=request,size = 100)['hits']['hits']
print(results)

[{'_index': 'emo31', '_type': 'emojis', '_id': 'nr2sFYUBDqVhmJIZoIO7', '_score': 1.0, '_source': {'index': {'_index': 'emo31'}}}, {'_index': 'emo31', '_type': 'emojis', '_id': 'n72sFYUBDqVhmJIZoIO7', '_score': 1.0, '_source': {'id': 0, 'emoji': '😀', 'meaning': ['grinning_face', 'face', 'smile', 'happy', 'joy', ':D', 'grin']}}, {'_index': 'emo31', '_type': 'emojis', '_id': 'oL2sFYUBDqVhmJIZoIO7', '_score': 1.0, '_source': {'index': {'_index': 'emo31'}}}, {'_index': 'emo31', '_type': 'emojis', '_id': 'ob2sFYUBDqVhmJIZoIO7', '_score': 1.0, '_source': {'id': 1, 'emoji': '😃', 'meaning': ['grinning_face_with_big_eyes', 'face', 'happy', 'joy', 'haha', ':D', ':)', 'smile', 'funny']}}, {'_index': 'emo31', '_type': 'emojis', '_id': 'or2sFYUBDqVhmJIZoIO7', '_score': 1.0, '_source': {'index': {'_index': 'emo31'}}}, {'_index': 'emo31', '_type': 'emojis', '_id': 'o72sFYUBDqVhmJIZoIO7', '_score': 1.0, '_source': {'id': 2, 'emoji': '😄', 'meaning': ['grinning_face_with_smiling_eyes', 'face', 'happy', '

/Users/anureddy/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:3: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


In [14]:
#ES search query for a specific word (testcase)
request = '''{"query": {"match": { "meaning":"kiss"}}}'''
results = client.search(index=index_name, body=request,size = 100)['hits']['hits']
print(results)

[{'_index': 'emo31', '_type': 'emojis', '_id': 'J72sFYUBDqVhmJIZoIW7', '_score': 6.634506, '_source': {'id': 196, 'emoji': '👄', 'meaning': ['mouth', 'mouth', 'kiss']}}, {'_index': 'emo31', '_type': 'emojis', '_id': 'R4anrIUBOSZGbDlKy07z', '_score': 6.634506, '_source': {'id': 196, 'emoji': '👄', 'meaning': ['mouth', 'mouth', 'kiss']}}, {'_index': 'emo31', '_type': 'emojis', '_id': 'db2sFYUBDqVhmJIZoIS7', '_score': 6.0449953, '_source': {'id': 107, 'emoji': '😽', 'meaning': ['kissing_cat', 'animal', 'cats', 'kiss']}}, {'_index': 'emo31', '_type': 'emojis', '_id': 'lYanrIUBOSZGbDlKy03z', '_score': 6.0449953, '_source': {'id': 107, 'emoji': '😽', 'meaning': ['kissing_cat', 'animal', 'cats', 'kiss']}}, {'_index': 'emo31', '_type': 'emojis', '_id': 'x72sFYUBDqVhmJIZoIO7', '_score': 5.132836, '_source': {'id': 20, 'emoji': '😙', 'meaning': ['kissing_face_with_smiling_eyes', 'face', 'affection', 'valentines', 'infatuation', 'kiss']}}, {'_index': 'emo31', '_type': 'emojis', '_id': '54anrIUBOSZGbDl

/Users/anureddy/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:2: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  


In [30]:
[d['_source']['meaning'] for d in results]

[['mouth', 'mouth', 'kiss'],
 ['mouth', 'mouth', 'kiss'],
 ['kissing_cat', 'animal', 'cats', 'kiss'],
 ['kissing_cat', 'animal', 'cats', 'kiss'],
 ['kissing_face_with_smiling_eyes',
  'face',
  'affection',
  'valentines',
  'infatuation',
  'kiss'],
 ['kissing_face_with_smiling_eyes',
  'face',
  'affection',
  'valentines',
  'infatuation',
  'kiss'],
 ['kiss', 'pair', 'valentines', 'love', 'like', 'dating', 'marriage'],
 ['kiss', 'pair', 'valentines', 'love', 'like', 'dating', 'marriage'],
 ['face_blowing_a_kiss',
  'face',
  'love',
  'like',
  'affection',
  'valentines',
  'infatuation',
  'kiss'],
 ['kissing_face',
  'love',
  'like',
  'face',
  '3',
  'valentines',
  'infatuation',
  'kiss'],
 ['kissing_face_with_closed_eyes',
  'face',
  'love',
  'like',
  'affection',
  'valentines',
  'infatuation',
  'kiss'],
 ['face_blowing_a_kiss',
  'face',
  'love',
  'like',
  'affection',
  'valentines',
  'infatuation',
  'kiss'],
 ['kissing_face',
  'love',
  'like',
  'face',
  '

In [34]:
"""remove stop words,
perform a word association based on twitter corpus.
perform ES query based on associated words.
return: corresponding word and associated emoji as list of tuple.
NOTE: not finished. """

def emoji_search(input_str):
    emoji_search_results = []
    en_stops = set(nltk.corpus.stopwords.words('english'))
    word_list = [x.strip() for x in input_str.split(" ")]
    model = Word2Vec.load('model.bin')
    for word in word_list:
        if word not in en_stops:
            # perform association
            association_list = model.wv.most_similar(word)
            # perform emoji search
            for word_associated in association_list:
            # TODO: search for emoji with keywords related
            # store in list_of_emoji
            # TODO: return word_associated with associated emoji
                emoji_search_results.append(word,list_of_emoji)
    return emoji_search_results
    # TODO: can also based on api to reversedictionary.com
    # which gives associated words to noun phrases: 
    # 1. extract noun phrases with nlp().noun_chunks,2.search
    # the phrase up in reversedictionary.com, and
    # 3.perform query with ES in emoji dataset with obtained 
    # set of words.

In [ ]:
"""The general function that would 
translate input to sentence with slang&emoji.
return&print translated sentence.
To be continued.
"""
def translation_style(input_str):
    